<div class="markdown-google-sans">

## **Badaga - English Transliteration Project**
## **t5**
</div>

In [ ]:
!pip install transformers[torch] sentencepiece accelerate datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_metric
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [ ]:
df = pd.read_excel('Badaga-v0.1.0.xlsx')
df.head()

,items,translated_transcript,audio_file_name,translterated_script,user_id,gender,locale,split_label,duration
0,1,what is the recipe in home,F002_1_1.mp3,manaya aena udhaka,F002,F,ba,train,2.377187
1,2,who is there near you,F002_1_2.mp3,pakka dhara edhdharae,F002,F,ba,train,2.377187
2,3,what did you prepare for lunch,F002_1_3.mp3,hagulu hasuga aena maditha,F002,F,ba,train,2.351062
3,4,did you brush,F002_1_4.mp3,hallu ujjithaya,F002,F,ba,train,2.194313
4,5,did you eat,F002_1_5.mp3,nee thindhubutaya,F002,F,ba,train,2.272687


In [ ]:
df.split_label.value_counts()

train         6897
validation    1470
test          1470
Name: split_label, dtype: int64

In [ ]:
# Dropping null values

df.dropna(inplace=True)
df.shape

(9834, 9)

In [ ]:
df.split_label.value_counts()

train         6895
validation    1470
test          1469
Name: split_label, dtype: int64

In [ ]:
# Splitting data into train, test and valid

train_df = df[df['split_label']=='train']
valid_df = df[df['split_label']=='validation']
test_df  = df[df['split_label']=='test']

In [ ]:
# Transforming Dataframes to pairs of translated(en) and transliterated text(ba)

def generate_pairs(df):
  eng = df['translated_transcript'].to_list()
  bad = df['translterated_script'].to_list()
  en_ba = []
  for i,j in zip(eng,bad):
    #en_ba.append({'translation':{'en':i,'ba':j}})
    en_ba.append({'en':i,'ba':j})
  return en_ba

train_pairs = generate_pairs(train_df)
valid_pairs = generate_pairs(valid_df)
test_pairs  = generate_pairs(test_df)

In [ ]:
train_pairs[0]

{'en': 'what is the recipe in home', 'ba': 'manaya aena udhaka'}

In [ ]:
train_pairs

[{'en': 'what is the recipe in home', 'ba': 'manaya aena udhaka'},
 {'en': 'who is there near you', 'ba': 'pakka dhara edhdharae'},
 {'en': 'what did you prepare for lunch', 'ba': 'hagulu hasuga aena maditha'},
 {'en': 'did you brush', 'ba': 'hallu ujjithaya'},
 {'en': 'did you eat', 'ba': 'nee thindhubutaya'},
 {'en': 'stand here for some time', 'ba': 'elli osi jaama nillu'},
 {'en': 'next year i will built a house',
  'ba': 'hadatha varusa na mannae kattinae'},
 {'en': 'next year i will come to village festival',
  'ba': 'hadatha varusa na habba ga bannae'},
 {'en': 'wait i will come', 'ba': 'bannae eru'},
 {'en': 'this bottle has more water',
  'ba': 'ei putti ya jaasthi neeru hadathae'},
 {'en': 'come early today', 'ba': 'endhdhu manakana ba'},
 {'en': 'i woke up so early in this moring',
  'ba': 'endhdhu orukuthu appara othina aedhubuttae'},
 {'en': 'i will wash my face and come', 'ba': 'na moga thogithu bannae'},
 {'en': 'see the time in your watch', 'ba': 'ninna gadigaara na man

In [ ]:
# Generating new dataframes from dictionary pairs

new_train_df = pd.DataFrame({'translation':train_pairs})
new_train_df = new_train_df.reset_index(drop=True)
new_train_df.to_csv("translation_train.csv", sep="\t", encoding="utf-8", index=False)

new_valid_df = pd.DataFrame({'translation':valid_pairs})
new_valid_df = new_valid_df.reset_index(drop=True)
new_valid_df.to_csv("translation_valid.csv", sep="\t", encoding="utf-8", index=False)

new_test_df  = pd.DataFrame({'translation':test_pairs})
new_test_df  = new_test_df.reset_index(drop=True)
new_test_df.to_csv("translation_test.csv", sep="\t", encoding="utf-8", index=False)

In [ ]:
from datasets import Dataset, DatasetDict

# Assuming you have train_pairs, valid_pairs, and test_pairs already defined as list of dictionaries

# Function to extract 'translation' part from each dictionary
def extract_translation(data):
    return {'translation': [d for d in data]}

# Convert train_pairs, valid_pairs, and test_pairs to Dataset
train_dataset = Dataset.from_dict(extract_translation(train_pairs))
valid_dataset = Dataset.from_dict(extract_translation(valid_pairs))
test_dataset = Dataset.from_dict(extract_translation(test_pairs))

# Create a DatasetDict
namma_data = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset,
    'test': test_dataset
})



In [ ]:
namma_data['train'][0]

{'translation': {'ba': 'manaya aena udhaka',
  'en': 'what is the recipe in home'}}

In [ ]:
# Loading csv files in Dataset format

data_files = {
    "train": "translation_train.csv",
    "validation": "translation_valid.csv",
    "test": "translation_test.csv"
}

raw_datasets  = load_dataset("csv", data_files=data_files, delimiter="\t", )
metric = load_metric("sacrebleu")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
raw_datasets['train'][0]

{'translation': "{'translation': {'en': 'what is the recipe in home', 'ba': 'manaya aena udhaka'}}"}

In [ ]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'tomorrow is a good day', 'ba': 'naaiga olleya jaenna'}"
1,"{'en': 'dont take too long come fast', 'ba': 'appara jaama maada baeda manakana baa'}"
2,"{'en': 'badaga songs will be good', 'ba': 'badaga paatu aella sooka attra'}"
3,"{'en': 'dont cry', 'ba': 'kaneeru haka beda'}"
4,"{'en': 'i have nine thousand rupees', 'ba': 'aenna sarae ombathu saavira uripi hadathae'}"


In [ ]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.5 MB/s eta 0:00:00


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
path = "/content/drive/MyDrive/t5-small-en-to-ba"


tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSeq2SeqLM.from_pretrained(path)

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = 'en'
target_lang = 'ba'
prefix = "Translate English to Badaga\n"
import ast

# def preprocess_function(examples):
#     inputs = [ast.literal_eval(ex)[source_lang] for ex in examples["translation"]]
#     targets = [ast.literal_eval(ex)[target_lang] for ex in examples["translation"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

#     labels = tokenizer(targets, max_length=max_target_length, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
for example in namma_data['train']['translation']:
    print(type(example))
    break

<class 'dict'>


In [ ]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [ ]:
raw_datasets['train'][3562]

{'translation': "{'en': 'hullathi is my mothers village', 'ba': 'hullathi enna amma na hatti'}"}

In [ ]:
tokenized_datasets = namma_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/6895 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1469 [00:00<?, ? examples/s]

In [ ]:
batch_size = 16

args = Seq2SeqTrainingArguments(
    "t5-small-en-to-ba",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    save_steps=100,
    logging_steps=10,
    predict_with_generate=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.758800,0.583870,45.548700,17.925200
2,0.737700,0.567932,45.845800,18.006800
3,0.742500,0.552359,46.392500,18.069400
4,0.699400,0.540336,46.896600,17.916300
5,0.812100,0.528964,47.266100,17.668700
6,0.609700,0.515833,47.752400,17.904100
7,0.641400,0.506239,48.602200,17.918400
8,0.708500,0.495140,48.690800,17.846900
9,0.595300,0.486213,49.784200,17.884400
10,0.649400,0.477346,49.949000,17.970700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 300, 'early_stopping': True, 'num_beams': 4}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 300, 'early_stopping': True, 'num_beams': 4}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

TrainOutput(global_step=12930, training_loss=0.6277354468694512, metrics={'train_runtime': 3955.3848, 'train_samples_per_second': 52.296, 'train_steps_per_second': 3.269, 'total_flos': 991778905718784.0, 'train_loss': 0.6277354468694512, 'epoch': 30.0})

In [ ]:
path = "/content/drive/MyDrive/t5-small-en-to-ba"

model.save_pretrained(path)
tokenizer.save_pretrained(path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 300, 'early_stopping': True, 'num_beams': 4}


('/content/drive/MyDrive/t5-small-en-to-ba/tokenizer_config.json',
 '/content/drive/MyDrive/t5-small-en-to-ba/special_tokens_map.json',
 '/content/drive/MyDrive/t5-small-en-to-ba/spiece.model',
 '/content/drive/MyDrive/t5-small-en-to-ba/added_tokens.json',
 '/content/drive/MyDrive/t5-small-en-to-ba/tokenizer.json')

In [ ]:
path = "/content/drive/MyDrive/t5-small-en-to-ba"


tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSeq2SeqLM.from_pretrained(path)

In [ ]:
test_pairs

[{'en': 'lets see what is going to happen',
  'ba': 'nadatharava aenthu nooduva'},
 {'en': 'come to village i will tell something important',
  'ba': 'hatti ga ba baevara aegina'},
 {'en': 'ther is a little water in this bottle',
  'ba': 'ei putti ya osi neeru tha hadathae'},
 {'en': 'did you buy a new vechile', 'ba': 'osa bandi esithiya'},
 {'en': 'i buyed a new shirt', 'ba': 'na ondhu osa kamisu esithae'},
 {'en': 'still how much time is left', 'ba': 'ennu aesu jaama hadathae'},
 {'en': 'today i want to complete this', 'ba': 'edhu endhdhu beriki haputhu'},
 {'en': 'this is in your hands', 'ba': 'edhu ninna kai na tha hadathae'},
 {'en': 'bus should come fast', 'ba': 'bandi manakana bantharae sari'},
 {'en': 'honey is too  sweet', 'ba': 'jaenu appara seeya'},
 {'en': 'he likes simbu', 'ba': 'emaga simbu tha edathara'},
 {'en': 'he acts very well', 'ba': 'emma ollenge nadichina'},
 {'en': 'i am aslo a good actor', 'ba': 'naanu ollenge nadichinae'},
 {'en': 'you should also act well', '

In [ ]:
test_pairs

[{'en': 'lets see what is going to happen',
  'ba': 'nadatharava aenthu nooduva'},
 {'en': 'come to village i will tell something important',
  'ba': 'hatti ga ba baevara aegina'},
 {'en': 'ther is a little water in this bottle',
  'ba': 'ei putti ya osi neeru tha hadathae'},
 {'en': 'did you buy a new vechile', 'ba': 'osa bandi esithiya'},
 {'en': 'i buyed a new shirt', 'ba': 'na ondhu osa kamisu esithae'},
 {'en': 'still how much time is left', 'ba': 'ennu aesu jaama hadathae'},
 {'en': 'today i want to complete this', 'ba': 'edhu endhdhu beriki haputhu'},
 {'en': 'this is in your hands', 'ba': 'edhu ninna kai na tha hadathae'},
 {'en': 'bus should come fast', 'ba': 'bandi manakana bantharae sari'},
 {'en': 'honey is too  sweet', 'ba': 'jaenu appara seeya'},
 {'en': 'he likes simbu', 'ba': 'emaga simbu tha edathara'},
 {'en': 'he acts very well', 'ba': 'emma ollenge nadichina'},
 {'en': 'i am aslo a good actor', 'ba': 'naanu ollenge nadichinae'},
 {'en': 'you should also act well', '

In [ ]:
test_example = test_pairs[900]

In [ ]:
test_example

{'en': 'did you write everything', 'ba': 'aella baruthu butaya'}

In [ ]:
text = "Translate English to Badaga\n"+test_example['en']
print(text)

Translate English to Badaga
did you write everything


In [ ]:
eg = {'translation':test_example}
eg

{'translation': {'en': 'still how much time is left',
  'ba': 'ennu aesu jaama hadathae'}}

In [ ]:
text = "'en':'"+test_example['en']+"'"
print(text)

'en':'come to village i will tell something important'


In [ ]:
from transformers import pipeline

translator = pipeline("translation", model=model,tokenizer=tokenizer)
res = translator(text)[0]['translation_text']
res

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


'nee ollenge edhdhae'

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
import torch

input_en = []
true_ba = []
pred_ba = []

def preprocess_function(examples):
    inputs = "Translate English to Badaga\n"+examples["en"]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).to(device)
    return model_inputs

tokenized_test_dataset = new_test_df['translation'].map(preprocess_function)

In [ ]:
for i in range(len(test_pairs)):
    print(i)
    test_example = test_pairs[i]
    with torch.no_grad():
        # Assuming 'input_ids' and 'attention_mask' are the required keys for tokenized input
        input_ids = tokenized_test_dataset[i]['input_ids']
        attention_mask = tokenized_test_dataset[i]['attention_mask']

        # Generate output
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask)

        # Decode output
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)

        # Append to lists
        input_en.append(test_example['en'])
        true_ba.append(test_example['ba'])
        pred_ba.append(decoded_output[0])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
for i in range(len(test_pairs)):
  print(i)
  test_example = test_pairs[i]
  text = "Translate English to Badaga\n"+test_example['en']
  res = translator(text)[0]['translation_text']
  input_en.append(test_example['en'])
  true_ba.append(test_example['ba'])
  pred_ba.append(res)

0


AttributeError: 'str' object has no attribute 'to'

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n=1000
test_example = valid_pairs[n]

true_ba = []
pred_ba = []

def preprocess_function(examples):
    inputs = examples[""]["en"]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).to(device)
    return model_inputs


tokenized_test_dataset = new_test_df['translation'].map(preprocess_function)
model.to(device)
# Inference
model.eval()
with torch.no_grad():
      output = model.generate(**tokenized_test_dataset[n])
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print("Original English Sentence:",test_example['en'])
print("Original Badaga Sentence:",test_example['ba'])
print("Predicted Badaga Sentence:", decoded_output)

KeyError: 'en'

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 1000

# Define preprocessing function
def preprocess_function(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    return inputs.to(device)

test_example = valid_pairs[n]

# Tokenize test example
tokenized_test_example = preprocess_function(test_example['en'])

# Model inference
model.to(device)
model.eval()
with torch.no_grad():
    output = model.generate(**tokenized_test_example)

# Decode output
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)

# Print results
print("Original English Sentence:", test_example['en'])
print("Original Badaga Sentence:", test_example['ba'])
print("Predicted Badaga Sentence:", decoded_output)


Original English Sentence: yesterday i felt so lazy
Original Badaga Sentence: nennae na soombaeri maaka edhdhae
Predicted Badaga Sentence: ['promote Israel promote am second Droing kindifferently 2016 am moram second Romania included ating included am second']


In [ ]:
n=10
test_example = valid_pairs[n]
src_text = test_example
input_ids = tokenizer(src_text['ba'], return_tensors="pt").to(device).input_ids
outputs = model.generate(input_ids, num_beams=5, num_return_sequences=3)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'recorded Prostrasetting ples Other OHnew ples new ples new Romania included ating included Other am second'

In [ ]:
t5_res_enba = pd.DataFrame({'input_en':input_en,'true_ba':true_ba,'pred_ba':pred_ba})
t5_res_enba

,input_en,true_ba,pred_ba
0,lets see what is going to happen,nadatharava aenthu nooduva,baasae hadathae nooduthuga
1,come to village i will tell something important,hatti ga ba baevara aegina,hatti ga ba baevara aegina
2,ther is a little water in this bottle,ei putti ya osi neeru tha hadathae,ei putti ya osi neeru tha hadathae
3,did you buy a new vechile,osa bandi esithiya,osa bandi esithiya
4,i buyed a new shirt,na ondhu osa kamisu esithae,na ondhu esithae esithae
...,...,...,...
1464,which fruit you like the most,ninnaga aena hannu appara edathara,ninnaga aena appara edathara
1465,i feel sleepy,aenaga orruku bapaningae hadathae,aenaga orruku bapaningae hadathae
1466,how many people are there in your house,ninga maennaya aesaga edhi,ninga manaya aesaga
1467,what is price for gold,chinna baellae aesaga vario,orruku baeku aena


In [ ]:
t5_res_enba.to_csv('t5_res_enba.csv')

In [ ]:
import sacrebleu

bleu_scores = [sacrebleu.raw_corpus_bleu([pred], [[true]]) for pred, true in zip(pred_ba, true_ba)]
bleu_scores_values = [score.score for score in bleu_scores]

# Compute average BLEU score
average_bleu_score = sum(bleu_scores_values) / len(bleu_scores_values)

print("Average BLEU Score:", average_bleu_score)

Average BLEU Score: 49.59464507035932


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.translate import meteor_score

# Tokenize hypotheses and references
tokenized_pred_ba = [pred.split() for pred in pred_ba]
tokenized_true_ba = [true.split() for true in true_ba]

# Calculate METEOR scores
meteor_scores = [meteor_score.meteor_score([true], pred) for pred, true in zip(tokenized_pred_ba, tokenized_true_ba)]

# Compute average METEOR score
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average METEOR Score:", average_meteor_score)

Average METEOR Score: 0.615183674890644


Badaga to English

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = 'ba'
target_lang = 'en'
prefix = "Translate Badaga to English\n"
import ast

# def preprocess_function(examples):
#     inputs = [ast.literal_eval(ex)[source_lang] for ex in examples["translation"]]
#     targets = [ast.literal_eval(ex)[target_lang] for ex in examples["translation"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

#     labels = tokenizer(targets, max_length=max_target_length, truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_datasets = namma_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/6895 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/1469 [00:00<?, ? examples/s]

In [ ]:
batch_size = 16

args = Seq2SeqTrainingArguments(
    "t5-small-ba-to-en",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    save_steps=100,
    logging_steps=10,
    predict_with_generate=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
metric = load_metric("sacrebleu")

<ipython-input-21-aede95515358>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.873900,0.765463,49.533700,7.118400
2,0.829800,0.739603,52.165500,7.119000
3,0.972100,0.717642,53.038800,7.025200
4,0.817600,0.689093,55.285500,7.087100
5,0.885200,0.671390,56.102300,7.134700
6,0.688100,0.650935,57.196100,7.146900
7,0.825100,0.632789,57.674500,7.154400
8,0.807900,0.614833,58.657800,7.108800
9,0.699700,0.603403,60.241900,7.085000
10,0.727400,0.582770,60.790800,7.153700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.873900,0.765463,49.533700,7.118400
2,0.829800,0.739603,52.165500,7.119000
3,0.972100,0.717642,53.038800,7.025200
4,0.817600,0.689093,55.285500,7.087100
5,0.885200,0.671390,56.102300,7.134700
6,0.688100,0.650935,57.196100,7.146900
7,0.825100,0.632789,57.674500,7.154400
8,0.807900,0.614833,58.657800,7.108800
9,0.699700,0.603403,60.241900,7.085000
10,0.727400,0.582770,60.790800,7.153700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=12930, training_loss=0.7156995044599092, metrics={'train_runtime': 2084.2073, 'train_samples_per_second': 99.246, 'train_steps_per_second': 6.204, 'total_flos': 2055860732461056.0, 'train_loss': 0.7156995044599092, 'epoch': 30.0})

In [ ]:
path = "/content/drive/MyDrive/t5-small-ba-to-en"

model.save_pretrained(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/t5-small-ba-to-en/tokenizer_config.json',
 '/content/drive/MyDrive/t5-small-ba-to-en/special_tokens_map.json',
 '/content/drive/MyDrive/t5-small-ba-to-en/spiece.model',
 '/content/drive/MyDrive/t5-small-ba-to-en/added_tokens.json',
 '/content/drive/MyDrive/t5-small-ba-to-en/tokenizer.json')

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
import torch

input_ba = []
true_en = []
pred_en = []

def preprocess_function(examples):
    inputs = "Translate Badaga to English\n"+examples["ba"]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).to(device)
    return model_inputs

tokenized_test_dataset = new_test_df['translation'].map(preprocess_function)

In [ ]:
i=895
test_example = test_pairs[i]
with torch.no_grad():
    # Assuming 'input_ids' and 'attention_mask' are the required keys for tokenized input
    input_ids = tokenized_test_dataset[i]['input_ids']
    attention_mask = tokenized_test_dataset[i]['attention_mask']

    # Generate output
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    # Decode output
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
    print("Original Badaga Sentence:",test_example['ba'])
    print("Original English Sentence:",test_example['en'])
    print("Predicted English Sentence:",decoded_output[0])
    # Append to lists
    input_ba.append(test_example['ba'])
    true_en.append(test_example['en'])
    pred_en.append(decoded_output[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original Badaga Sentence: aenna sarae munoora haaru uripi hadathae
Original English Sentence: i have three hundred and six rupees
Predicted English Sentence: i have three hundred and six rupees


In [ ]:
input_ba = []
true_en = []
pred_en = []

for i in range(len(test_pairs)):
    print(i)
    test_example = test_pairs[i]
    with torch.no_grad():
        # Assuming 'input_ids' and 'attention_mask' are the required keys for tokenized input
        input_ids = tokenized_test_dataset[i]['input_ids']
        attention_mask = tokenized_test_dataset[i]['attention_mask']

        # Generate output
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask)

        # Decode output
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)

        # Append to lists
        input_ba.append(test_example['ba'])
        true_en.append(test_example['en'])
        pred_en.append(decoded_output[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
t5_res_baen = pd.DataFrame({'input_ba':input_ba,'true_en':true_en,'pred_en':pred_en})
t5_res_baen

,input_ba,true_en,pred_en
0,nadatharava aenthu nooduva,lets see what is going to happen,lets see and watch
1,hatti ga ba baevara aegina,come to village i will tell something important,tell me a story
2,ei putti ya osi neeru tha hadathae,ther is a little water in this bottle,this bottle contains less water
3,osa bandi esithiya,did you buy a new vechile,did you buy a new vechile
4,na ondhu osa kamisu esithae,i buyed a new shirt,i have bought one for you
...,...,...,...
1464,ninnaga aena hannu appara edathara,which fruit you like the most,what do you like the moist
1465,aenaga orruku bapaningae hadathae,i feel sleepy,i feel sleepy
1466,ninga maennaya aesaga edhi,how many people are there in your house,how many people are there in your house
1467,chinna baellae aesaga vario,what is price for gold,what is the price for onion


In [ ]:
t5_res_baen.to_csv('t5_res_baen.csv')

In [ ]:
import sacrebleu

bleu_scores = [sacrebleu.raw_corpus_bleu([pred], [[true]]) for pred, true in zip(pred_en, true_en)]
bleu_scores_values = [score.score for score in bleu_scores]

# Compute average BLEU score
average_bleu_score = sum(bleu_scores_values) / len(bleu_scores_values)

print("Average BLEU Score:", average_bleu_score)

Average BLEU Score: 68.95599911043843


In [ ]:
from nltk.translate import meteor_score

# Tokenize hypotheses and references
tokenized_pred_ba = [pred.split() for pred in pred_en]
tokenized_true_ba = [true.split() for true in true_en]

# Calculate METEOR scores
meteor_scores = [meteor_score.meteor_score([true], pred) for pred, true in zip(tokenized_pred_ba, tokenized_true_ba)]

# Compute average METEOR score
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average METEOR Score:", average_meteor_score)

Average METEOR Score: 0.767129459673514
